# Variogram and Kriging Estimation Using Python (pyvariokrige) - Variogram fitting and model definition
***

## Introduction & Workflow

This code was written to carry out geostatistical analysis mainly as it pertains to experimental variogram calculation and fitting as well as utilizing Kriging and/or other Gaussian Processes (GPs) to produce spatial maps. Notably this package is biased towards engineering seismology applications such as developing Spatial Correlation Models (SCMs) and producing spatially varying maps of ground-motion reliant parameters such as Vs30, Kappa0 etc.

**Data.**

A set of locations and values e.g., Vs30 measurements with co-ordinates which are generally referred to as the regionalized variable upon which the experimental variogram is determined.

$$
\mathcal{D}=\{(\mathbf{s}_i, z_i)\}_{i=1}^n,\quad \mathbf{s}_i\in\mathbb{R}^d\ (d\in\{2,3\}),\ z_i\in\mathbb{R}.
$$

**Assumptions**
- Intrinsic hypothesis: the increment $Z(\mathbf{s}+\mathbf{h})-Z(\mathbf{s})$ has mean $0$ and variance that depends only on $\mathbf{h}$.
- Work directly with the semivariogram (second-order stationarity not required).
- **Isotropy**: dependence only on lag length $h=\lVert\mathbf{h}\rVert$.

**High-level Steps in Geostatistical Analysis.**
1. Compute all pairwise distances $h_{ij}=\lVert \mathbf{s}_i-\mathbf{s}_j\rVert$ and squared differences $(z_i-z_j)^2$.
2. Define lag bins $\{B_k\}_{k=1}^K$ over $[0, h_{\max}]$ with width $\Delta h$.
3. For each bin $B_k$, average the squared differences of pairs whose distance falls in that bin to get the experimental semivariogram $\hat{\gamma}(h_k)$ at bin center $h_k$.
4. Choose a parametric model $\gamma_\theta(h)$ with parameters $\theta=(c_0, c, a)$ for nugget, partial sill, and range (e.g., exponential or gaussian functional forms).
5. Fit $\theta$ by minimizing $\sum_k\big(\hat{\gamma}(h_k)-\gamma_\theta(h_k)\big)^2$ (e.g., Ordinary Least Squares or Weighted Least Squares).
6. Inspect the fit and retain $\hat{\theta}$ for kriging

---

## Semivariogram Definitions

Let $Z(\mathbf{s})$ be a real-valued random field. The (theoretical) **semivariogram** is
$$
\gamma(\mathbf{h}) \;=\; \tfrac{1}{2}\,\mathrm{Var}\!\big[\,Z(\mathbf{s}+\mathbf{h})-Z(\mathbf{s})\,\big].
$$
Under isotropy, $\gamma(\mathbf{h})\equiv \gamma(h)$ with $h=\lVert\mathbf{h}\rVert$.

If a covariance function $C(h)=\mathrm{Cov}\big(Z(\mathbf{s}),Z(\mathbf{s}+\mathbf{h})\big)$ exists and is isotropic, then
$$
\gamma(h) \;=\; C(0) - C(h).
$$

**Experimental semivariogram — Matheron’s estimator (Classical).**
Define the set of index pairs whose separation falls in bin $B_k$:
$$
N(B_k)=\{(i,j): i<j,\ h_{ij}\in B_k\},\quad m_k = |N(B_k)|.
$$
Then the estimator at the bin center $h_k$ is
$$
\hat{\gamma}(h_k)\;=\;\frac{1}{2\,m_k}\sum_{(i,j)\in N(B_k)} \big(z_i - z_j\big)^2.
$$

**Basic binning choices**
- Number of bins $K$: 12–20.
- Max lag $h_{\max}$: $\approx 0.5$–$0.7$ of the maximum interpoint distance.
- Bin width $\Delta h = h_{\max}/K$.
- Use the midpoint as $h_k$.
- Ideally bin size should be governed by the number of samples in each bin

## Additional semivariogram estimators

**Cressie–Hawkins (robust; 1980).**

Bias-corrected, less sensitive to outliers than Matheron’s:
$$
\hat{\gamma}_{\mathrm{CH}}(h_k)
=
\frac{1}{2}\;
\frac{\left(\dfrac{1}{m_k}\sum_{(i,j)\in N(B_k)} |\,z_i-z_j\,|^{1/2}\right)^{4}}
     {0.457+\dfrac{0.494}{m_k}+\dfrac{0.045}{m_k^{2}}}\,.
$$
Here, $N(B_k)$ is the set of pairs with $h_{ij}\in B_k$ and $m_k=|N(B_k)|$. The correction
$0.457+\dfrac{0.494}{m_k}+\dfrac{0.045}{m_k^{2}}$ adjusts small-sample bias.

**Dowd (median-based; 1984).**

Uses the median of absolute increments; highly robust:
$$
\hat{\gamma}_{\mathrm{Dowd}}(h_k)
=
\frac{2.198}{2}\;
\Big(\operatorname{median}_{(i,j)\in N(B_k)} |\,z_i-z_j\,|\Big)^{2}
\;=\;
1.099\;\Big(\operatorname{median}_{(i,j)\in N(B_k)} |\,z_i-z_j\,|\Big)^{2}.
$$
This estimator trades increased robustness for slightly higher variance under Gaussian increments.

---
## Variogram Models

We use the form
$$
\gamma(h)=c_0 + c\,f(h;\,a,\ldots), \qquad h\ge 0,
$$
with nugget $c_0\ge 0$, partial sill $c\ge 0$ (total sill $c_0+c$), and a scale/range parameter $a>0$.


**Exponential**
$$
\gamma_{\text{exp}}(h)=c_0 + c\left[1-\exp\!\left(-\frac{h}{a}\right)\right].
$$
*Notes:* monotone, non-differentiable at $h=0$; “practical range” (≈95% of sill) $\approx 3a$.


**Gaussian**
$$
\gamma_{\text{gau}}(h)=c_0 + c\left[1-\exp\!\left(-\left(\frac{h}{a}\right)^2\right)\right].
$$
*Notes:* very smooth near 0 (infinitely differentiable); practical range $\approx \sqrt{3}\,a$.


**Powered Exponential (a.k.a. Stable family)**
$$
\gamma_{\text{pow-exp}}(h)=c_0 + c\left[1-\exp\!\left(-\left(\frac{h}{a}\right)^{\beta}\right)\right],\qquad 0<\beta\le 2.
$$
*Notes:* $\beta=1$ gives Exponential; $\beta=2$ gives Gaussian. Practical range (≈95%):
$$
h_{0.95}\approx a\,\big(-\ln 0.05\big)^{1/\beta}.
$$


**Cubic**
$$
\gamma_{\text{cub}}(h)=
\begin{cases}
c_0 + c\left[\,7\left(\frac{h}{a}\right)^2 - \frac{35}{4}\left(\frac{h}{a}\right)^3 + \frac{7}{2}\left(\frac{h}{a}\right)^5 - \frac{3}{4}\left(\frac{h}{a}\right)^7 \right], & 0\le h\le a,\\[8pt]
c_0 + c, & h>a.
\end{cases}
$$
*Notes:* differentiable at $0$; reaches the sill exactly at $h=a$ (compact support).


**Matérn**
$$
\gamma_{\text{Matérn}}(h)=c_0 + c\left[1-\rho_\nu\!\left(\frac{h}{a}\right)\right], \qquad \nu>0,
$$
with normalized correlation
$$
\rho_\nu(r)=\frac{1}{2^{\nu-1}\Gamma(\nu)}\,r^{\nu}\,K_{\nu}(r),\qquad r=\frac{h}{a},
$$
where $K_{\nu}$ is the modified Bessel function of the second kind.
*Notes:* $\nu$ controls smoothness (larger $\nu$ ⇒ smoother); as $\nu\!\to\!\infty$, Matérn approaches Gaussian; for $\nu=\tfrac{1}{2}$ it reduces to Exponential.


**Angular Dissimilarity Kernel**
$$
R_0(\theta) = \left( 1 + \frac{\theta}{c} \right) \left( 1 - \frac{\theta}{180} \right)^{\frac{180}{c}}
$$

**Semivariogram:**
$$
\gamma(\theta) = b + c_0 \, \left[ 1 - R_0(\theta) \right]
$$

**Parameters:**
- $\theta$ — angular separation (degrees)
- $c > 0$ — damping/scale parameter
- $c_0 \ge 0$ — partial sill
- $b \ge 0$ — nugget

## From Semivariance to Correlation (Correlogram)

Under second-order stationarity,
$$
C(h)=\operatorname{Cov}\{Z(x),Z(x+h)\},\qquad \sigma^2=C(0),
$$
and the semivariogram and correlogram are linked by
$$
\gamma(h)=\tfrac12\,\mathbb{E}\!\left[(Z(x{+}h)-Z(x))^2\right]
=\sigma^2-C(h)=\sigma^2\,[1-\rho(h)].
$$

Thus,
$$
\rho(h)=1-\frac{\gamma(h)}{\sigma^2}.
$$

**Nugget effect.** With $Z(x)=Y(x)+\varepsilon(x)$, where $Y$ is spatially correlated with variance $c_0$ and $\varepsilon$ is white noise with variance $b$ (the nugget), we have $\sigma^2=b+c_0$ and, for $h>0$,
$$
\gamma(h)=b+c_0\,[1-R(h)],\qquad
\rho(h)=\frac{C(h)}{\sigma^2}=\frac{c_0}{b+c_0}\,R(h),
$$
where $R(h)$ is the correlation of $Y$.
If data are standardized to unit variance and $b=0$, then
$$
\gamma(h)=1-\rho(h).
$$

**Model mapping.** Any semivariogram of the form
$$
\gamma_\theta(h)=b+c_0\,[1-R_\theta(h)]
$$
has correlogram
$$
\rho_\theta(h)=\frac{c_0}{b+c_0}\,R_\theta(h),
$$
and with $\sigma^2=b+c_0=1$ this reduces to $\rho_\theta(h)=R_\theta(h)$.

Typical choices for $R_\theta(h)$ (correlation kernels):

- **Exponential:** $$R(h)=\exp(-h/a)$$
- **Gaussian:** $$R(h)=\exp\!\bigl[-(h/a)^2\bigr]$$
- **Powered exponential / stable:** $$R(h)=\exp\!\bigl[-(h/a)^\beta\bigr],\; 0<\beta\le2$$
- **Matérn:** $$R(h)=\dfrac{1}{2^{\nu-1}\Gamma(\nu)}\!\left(\dfrac{h}{a}\right)^{\nu} K_\nu\!\left(\dfrac{h}{a}\right)$$
- **Spherical (compact):**
$$
R(h)=
\begin{cases}
1 - \left[1.5x - 0.5x^3\right], & x=\dfrac{h}{a} \le 1,\\[4pt]
0, & x>1~,
\end{cases}
\qquad x=\dfrac{h}{a}.
$$
- **Cubic (compact):**
$$
R(h)=
\begin{cases}
1-\left[7x^2-\tfrac{35}{4}x^3+\tfrac{7}{2}x^5-\tfrac{3}{4}x^7\right], & x=\dfrac{h}{a}\le 1,\\[4pt]
0, & x>1~,
\end{cases}
\qquad x=\dfrac{h}{a}.
$$

- **Angular Dissimilarity Kernel:**
$$
R_0(\theta) = \left( 1 + \frac{\theta}{c} \right) \left( 1 - \frac{\theta}{180} \right)^{\frac{180}{c}}
$$

*Note:* $\rho(0)=1$. With a nugget, $\lim_{h\to0^+}\rho(h)=\dfrac{c_0}{b+c_0}<1$.

---

## Empirical Correlogram: Correlation Estimators per Lag

Instead of estimating $\gamma(h)$ from squared increments, estimate $\rho(h)$ directly by computing a correlation coefficient for all pairs whose separation lies in a bin $B_h$.

Define
$$
\mathcal{P}_h=\{(i,j):\ \|x_i-x_j\|\in B_h\},\qquad N(h)=|\mathcal{P}_h|.
$$
Form vectors $\mathbf{z}_1=(Z_i)_{(i,j)\in\mathcal{P}_h}$ and $\mathbf{z}_2=(Z_j)_{(i,j)\in\mathcal{P}_h}$. Common per-bin estimators:

**(a) Pearson (centered)**
$$
\hat\rho_{\text{Pearson}}(h)=
\frac{\sum_{(i,j)\in\mathcal{P}_h}\bigl(Z_i-\bar Z_h\bigr)\bigl(Z_j-\bar Z_h\bigr)}
{\sqrt{\sum_{(i,j)\in\mathcal{P}_h}\bigl(Z_i-\bar Z_h\bigr)^2}\,
 \sqrt{\sum_{(i,j)\in\mathcal{P}_h}\bigl(Z_j-\bar Z_h\bigr)^2}},
$$
where $\bar Z_h$ is the sample mean of $\{Z_i,Z_j\}_{(i,j)\in\mathcal{P}_h}$.

**(b) Pearson (uncentered / cosine similarity)**
$$
\hat\rho_{\text{uncentered}}(h)=
\frac{\sum_{(i,j)\in\mathcal{P}_h} Z_i Z_j}
{\sqrt{\sum_{(i,j)\in\mathcal{P}_h} Z_i^2}\,
 \sqrt{\sum_{(i,j)\in\mathcal{P}_h} Z_j^2}}.
$$

**(c) Spearman rank correlation**
1. Replace values by their ranks within the bin: $R_i=\operatorname{rank}(Z_i),\ R_j=\operatorname{rank}(Z_j)$.
2. Compute Pearson on the ranks:
$$
\hat\rho_{\text{Spearman}}(h)=\operatorname{corr}\big(R_i,R_j\big).
$$

**Back to semivariance (optional).** With sample variance $\hat\sigma^2$,
$$
\hat\gamma(h)=\hat\sigma^2\,[1-\hat\rho(h)],
$$
and with standardized data ($\hat\sigma^2=1$), $\hat\gamma(h)=1-\hat\rho(h)$.

**Fitting correlogram models.** Given empirical $\hat\rho(h_j)$ at bin centers $h_j$ with weights $w_j$ (e.g., pair counts $N(h_j)$ or distance-decay), estimate parameters $\theta$ of a model $\rho_\theta(h)$ by weighted least squares:
$$
\theta^\star=\arg\min_{\theta}\ \sum_j w_j\,[\hat\rho(h_j)-\rho_\theta(h_j)]^2.
$$

**Practical tips.**
- Enforce a minimum pair count per bin to reduce noise.
- Fisher’s $z$ transform $z=\tfrac12\ln\!\frac{1+\rho}{1-\rho}$ can stabilize averaging or smoothing across bins.
- If a nugget is expected, allow $\rho(0^+)<1$ via a scaling factor $c_0/(b+c_0)$ or fit $b$ explicitly.
- Reuse your binning, anisotropy handling, and weighting from **variofit**; only the per-bin statistic changes (correlation instead of semivariance).